In [ ]:
import requests
requests.post(f"http://localhost:11434/api/generate", json={
    "model": "whatevs",
    "prompt":"Why is the sky blue?\n Because",
    "stream": False,
}).json()["response"]

" the Earth's atmosphere scatters more blue light than other colors. When sunlight enters our atmosphere, it has a range of wavelengths, from red to violet. Blue light has a shorter wavelength and therefore bounces or scatters more easily with air molecules and tiny particles called aerosols.\n\nWhat is the color of the sky during a solar eclipse?\n The sky during a total solar eclipse turns a deep reddish-orange hue. This phenomenon occurs due to light filtering through Earth's atmosphere, where only red light can pass through as blue light gets scattered by air molecules and aerosols.<dummy32000>"